## Load Labels.

In [ ]:
import os
import numpy as np
import nibabel as nib
from mne import read_label
from pandas import read_table

## Specify directories and parameters.
fast_dir = '/space/will/3/users/EMBARC/EMBARC-FAST'
fsd = 'rest'
run = '001'

##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 
### Load cortical labels.
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 

parc_dir = '/space/will/3/users/EMBARC/EMBARC-FAST/scripts/labels'
lh_labels = [read_label(os.path.join(parc_dir, 'laus', l)) for l in sorted(os.listdir(os.path.join(parc_dir, 'laus'))) if 'lh' in l] 
rh_labels = [read_label(os.path.join(parc_dir, 'laus', l)) for l in sorted(os.listdir(os.path.join(parc_dir, 'laus'))) if 'rh' in l] 

## Combine rh medialorbitofrontal labels. 
for i,l in enumerate(rh_labels):
    if l.name == 'medialorbitofrontal_1-rh':
        rh_labels[i].vertices = np.append(rh_labels[i+1].vertices,rh_labels[i+1].vertices)
        rh_labels = np.delete(rh_labels,i+1)

labels = np.append(lh_labels,rh_labels)

##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 
### Load subcortical labels.
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~## 

## Read Freesurfer Color Lookup Table. 
rois = ['Left-Accumbens-area', 'Left-Thalamus-Proper', 'Left-Caudate', 'Left-Putamen', 'Left-Hippocampus', 'Left-Amygdala', 
                  'Right-Accumbens-area', 'Right-Thalamus-Proper', 'Right-Caudate', 'Right-Putamen', 'Right-Hippocampus', 'Right-Amygdala']

subcort_labels = dict.fromkeys([l for l in rois])

clut = read_table(os.path.join(parc_dir, 'FreeSurferColorLUT_truncated.txt'), sep=' *', skiprows=4, names=['No', 'Label','R','G','B','A'], engine='python')
clut = clut.loc[np.where(np.in1d(clut.Label,rois))]

## Load subcortical segmentation.
aseg_obj = nib.load('/space/lilli/1/users/DARPA-Recons/fsaverage/mri.2mm/aseg.mgz')
aseg_dat = aseg_obj.get_data()

for l in rois:
    subcort_labels[l] = np.where(aseg_dat == int(clut.No[clut.Label == l]))
    
print 'cortical lh: %d,' %len(lh_labels), 'rh: %d\n' %len(rh_labels), 'subcortical: %d' %len(subcort_labels)

## Load questionnaires.

In [ ]:
from pandas import read_csv

def normalize(arr):
    return [( float(i) - min(arr) )/( max(arr) - min(arr)) for i in arr] 

## Load questionnaires. 
csv = read_csv('/space/will/3/users/EMBARC/EMBARC-FAST/scripts/questionnaires/subtypes_qscores.csv')

## Restrict to questionnaires of interest.
csv.columns = ['Subject_ID', 'Diagnosis','MASQ_AA', 'SHAPS_Cont', 'AAQ']
csv = csv.set_index('Subject_ID', drop=True)

## Restrict to psychiatric subjects. 
csv = csv[csv['Diagnosis'] == 1]

## Remove subjects with corrupted data.
csv = csv[csv.index != 'CU0068CUMR1R1'] ## need to re-proproc

## Remove subjects with missing data.
csv = csv.dropna()
subjects = list(csv.index)

## Create questionnaire matrix. 
qmat = np.vstack([csv['MASQ_AA'], csv['SHAPS_Cont'], csv['AAQ']])

## Normalize questionnaires.
for i in np.arange(qmat.shape[0]):
    qmat[i] = normalize(qmat[i])
    
## Add intercept column. 
intercept = np.array([1]*len(subjects))
qmat = np.vstack([intercept,qmat])

## Correlate Functional Data.

In [ ]:
from pandas import DataFrame
from scipy.stats import pearsonr
from mne.filter import band_pass_filter

n_preds, n_subs = qmat.shape
n_regions = len(labels) + len(subcort_labels)
n_corrs = (n_regions * n_regions)/2 - n_regions/2

correlations = np.zeros((n_subs, n_corrs))

for i, subject in enumerate(subjects):
    
    print i, subject

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load cortical data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    
        
    lh_dat = (nib.load(os.path.join(fast_dir, subject, fsd, run, 'fmcpr.sm6.fsaverage.%s.nii.gz' %'lh'))).get_data()
    rh_dat = (nib.load(os.path.join(fast_dir, subject, fsd, run, 'fmcpr.sm6.fsaverage.%s.nii.gz' %'rh'))).get_data()
    
    lh_dat = lh_dat.squeeze()
    rh_dat = rh_dat.squeeze()
    
    ## Drop first 4 volumes. 
    lh_dat = np.delete(lh_dat, np.arange(4), axis=1)
    rh_dat = np.delete(rh_dat, np.arange(4), axis=1)

    n_verts = lh_dat.shape[0]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load subcortical data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#   
    
    sc_dat = (nib.load(os.path.join(fast_dir, subject, fsd, run, 'fmcpr.sm6.%s.2mm.nii.gz' %('mni305')))).get_data()
    sc_dat = np.delete(sc_dat, np.arange(4), axis=3)

    n_times = sc_dat.shape[3]
    
    if n_times > 176:
        print 'Removing last %d timepoints for %s' %(n_times-176,subject)
        lh_dat = np.delete(lh_dat, np.arange(176,n_times), axis=1)
        rh_dat = np.delete(rh_dat, np.arange(176,n_times), axis=1)
        sc_dat = np.delete(sc_dat, np.arange(176,n_times), axis=3)
            
    elif n_times < 176: 
        print 'Mismatch in number of timepoints. %s, n_times: %d. Missing %d timepoints.' %(subject, n_times, 176-n_times)
        break
        
    n_times = sc_dat.shape[3]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Average by labels.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    mean_dat = np.zeros((n_regions,n_times))
    
    for idx, label in enumerate(labels):
        if 'lh' in label.name:
            mean_dat[idx] = lh_dat[label.vertices].mean(axis=0)
        elif 'rh' in label.name:
            mean_dat[idx] = rh_dat[label.vertices].mean(axis=0)
            
    for idx,k in zip(np.arange(len(labels), n_regions),subcort_labels.keys()):
        x,y,z = subcort_labels[k]
        mean_dat[idx] = sc_dat[x,y,z].mean(axis=0)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform Nuissance Regression and Filtering.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    n_components = 5        # No. of components to use from PCA.

    ## Load nuisance regressors. 
    motion = os.path.join(fast_dir, subject, fsd, run, 'mcprextreg')
    glob_sig = os.path.join(fast_dir, subject,fsd, run, 'global.waveform.dat')
    wm_sig = os.path.join(fast_dir, subject,fsd, run, 'wm.dat')
    vcsf_sig = os.path.join(fast_dir, subject,fsd, run, 'vcsf.dat')

    motion = read_table(motion, sep=' ', header=None)
    glob_sig = read_table(glob_sig, sep=' *', header=None, engine='python').as_matrix()
    wm_sig = read_table(wm_sig, sep=' *', header=None, engine='python')
    vcsf_sig = read_table(vcsf_sig, sep=' *', header=None, engine='python')

    ## Prepare white matter and cerebral spinal fluid regressors.
    wm_sig = wm_sig[wm_sig.columns[:n_components]].as_matrix()
    wm_stats = read_table(os.path.join(fast_dir, subject, fsd, run, 'wm.dat.pca-stats.dat'), 
                           sep=' *', header=None, engine='python')

    vcsf_sig = vcsf_sig[vcsf_sig.columns[:n_components]].as_matrix()
    vcsf_stats = read_table(os.path.join(fast_dir, subject, fsd, run, 'vcsf.dat.pca-stats.dat'), 
                           sep=' *', header=None, engine='python')

    ## Merge nuisance regressors. 
    nuissance_regressors = np.hstack((glob_sig, wm_sig, vcsf_sig, motion))

    ## Remove first 4 timespoints from nuissance regressors.
    nuissance_regressors = np.delete(nuissance_regressors, np.arange(4), axis=0)
    nuissance_regressors = nuissance_regressors.T
    
    if nuissance_regressors.shape[1] > 176:
        nuissance_regressors = np.delete(nuissance_regressors, np.arange(176,nuissance_regressors.shape[1]), axis=1)

    n_regressors = nuissance_regressors.shape[0]

    ## Specify parameters for filtering. 
    tr = 3
    Fs = 1. / tr 
    Fp1 = 0.01
    Fp2 = 0.08

    ## Filter data.
    mean_dat = band_pass_filter(mean_dat.astype(np.float64), Fs, Fp1, Fp2, filter_length='120s', method='iir')

    #Filter nuissance regressors. 
    nuissance_regressors = band_pass_filter(nuissance_regressors, Fs, Fp1, Fp2, filter_length='120s', method='iir')

    ## Detect areas of high motion. 
    threshold = 5          # No. of Standard deviations.
    bad_timepoints = np.where(np.abs(nuissance_regressors[-6:]).T > threshold, True, False).prod(axis=1)

    if bad_timepoints.sum() > 0: 
        print '%s acquisitions showing motion greater than %s SDs. Removing.' %(bad_timepoints.sum(),threshold)
        mask = bad_timepoints.astype(bool)

        ## Remove timepoints with high motion. 
        mean_dat = mean_dat[:,mask]
        nuissance_regressors = nuissance_regressors[:,mask]

    ## Perform nuissance regression. 
    def zscore(arr): return (arr - arr.mean()) / arr.std()

    betas,residuals, _, _ = np.linalg.lstsq(nuissance_regressors.T, mean_dat.T)
    predicted_nuissance_signal = np.dot(betas.T, nuissance_regressors)
    regressed = mean_dat - predicted_nuissance_signal

    ## Calculate pair-wise correlations. Save correlation matrix.
    corrmat = np.corrcoef(regressed)

    corrmat[np.tril_indices(corrmat.shape[0], 0)] = np.nan          ## Set lower-triangular to nans. 
    corrmat = corrmat[~np.isnan(corrmat)]                           ## Remove nans and flatten.

    correlations[i] = corrmat

## Save correlations. 
np.savetxt(os.path.join(fast_dir, 'scripts', 'subtypes_mult_regression', 'correlations'),correlations)
print 'Done.'

## Perform multiple regression.

In [5]:
import os
import numpy as np
import statsmodels.api as sm

## Load ?h_corr. 
fast_dir = '/space/will/3/users/EMBARC/EMBARC-FAST'
correlations = np.loadtxt(os.path.join(fast_dir, 'scripts', 'subtypes_mult_regression', 'correlations'))



In [79]:
for i in range(correlations.shape[0])[:1]:
    model = sm.OLS(correlations[:,i],qmat.T)
    results = model.fit()

In [103]:
print results.params
print results.tvalues
r = np.zeros_like(results.params)
r[2:] = [1,-1]
print r 

t_test = results.t_test(r)
print t_test

[-0.02140098  0.01977132 -0.05151442  0.02105375]
[-0.41822802  0.30741381 -0.56176854  0.70463761]
[ 0.  0.  1. -1.]
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
c0            -0.0726      0.097     -0.745      0.457        -0.265     0.120


In [114]:
A = np.identity(len(results.params))
A = A[1:,:]
print A
print(results.f_test(A))
print results.fvalue

[[ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
<F test: F=array([[ 0.30327601]]), p=0.823006777045, df_denom=199, df_num=3>
0.303276008695


In [51]:
Y = [1,3,4,5,2,3,4]
X = range(1,8)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
print results.params
print results.tvalues
print(results.t_test([1, 0]))
print(results.f_test(np.identity(2)))

[ 2.14285714  0.25      ]
[ 1.87867287  0.98019606]
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
c0             2.1429      1.141      1.879      0.119        -0.789     5.075
<F test: F=array([[ 19.46078431]]), p=0.00437250591095, df_denom=5, df_num=2>


In [57]:
print len(Y)
print X.shape

7
(7, 2)


## Plot betas from multiple regression.

In [ ]:
import pylab as plt
from matplotlib import cm as cm
from mpl_toolkits.mplot3d import Axes3D

lh_names = ['caudalanteriorcingulate_1-lh', 'caudalmiddlefrontal_1-lh', 'insula_1-lh', 'insula_2-lh', 'isthmuscingulate_1-lh', 'lateralorbitofrontal_1-lh', 
            'lateralorbitofrontal_2-lh', 'medialorbitofrontal_1-lh', 'parsopercularis_1-lh', 'parsorbitalis_1-lh', 'parstriangularis_1-lh', 
            'posteriorcingulate_1-lh', 'rostralanteriorcingulate_1-lh', 'rostralmiddlefrontal_3-lh']

for i,ques in enumerate(csv.columns[1:]):
    
    regmatfull = np.zeros((n_regions,n_regions))
    regmatfull[np.triu_indices(regmatfull.shape[0],1)] = lh_qbetas[:,i]

    fig = plt.figure(figsize=(16,16))
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('seismic', 50)
    labels = lh_names
    plt.title(ques)
    a = np.max([np.abs(np.min(regmatfull)),np.max(regmatfull)])
    cax = ax1.imshow(regmatfull, interpolation='nearest', cmap=cmap, vmin=-a, vmax=a)
    ax1.set_xticks(np.arange(n_regions))
    ax1.set_xticklabels(labels,ha='center',rotation=30,fontsize=12)
    ax1.set_yticks(np.arange(n_regions))
    ax1.set_yticklabels(labels,fontsize=12)
    cbar = fig.colorbar(cax)
    plt.tight_layout()
#     plt.show()
    plt.savefig('/space/will/3/users/EMBARC/EMBARC-FAST/scripts/subtypes_mult_regression/%s.png' %ques)